<a href="https://colab.research.google.com/github/sumedhakoranga/employee_future_prediction/blob/main/random_forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dataset link: https://www.kaggle.com/tejashvi14/employee-future-prediction

# Uploading dataset

In [ ]:
from google.colab import files

uploaded = files.upload()

Saving Employee.csv to Employee.csv


# Initialization

In [ ]:
import pandas as pd
import numpy as np

from itertools import product

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier

import warnings
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv('Employee.csv')

X = df.drop(['LeaveOrNot'], axis=1)
y = df['LeaveOrNot']

# Preparing data

In [ ]:
X_full_train, X_test, y_full_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_full_train, y_full_train, test_size=0.25, random_state=42)

In [ ]:
numerical = ['Age']
categorical = ['Education', 'JoiningYear', 'City', 'PaymentTier', 'Gender', 'EverBenched', 'ExperienceInCurrentDomain']

# Creating a Pipeline

In [ ]:
def create_new_pipeline(params):
    numerical_transformer = SimpleImputer(strategy='median')

    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('encoding', OneHotEncoder(drop='first'))
    ])

    preprocessor = ColumnTransformer(
    transformers=[
        ('numerical', numerical_transformer, numerical),
        ('categorical', categorical_transformer, categorical)
    ])

    model = RandomForestClassifier(
        n_jobs=-1,
        random_state=42,
        **params
    )

    pipeline = Pipeline(
    steps=[
           ('preprocessing', preprocessor),
           ('model', model)
          ]
    )

    return pipeline

# Hyperparameter Tuning

In [ ]:
search_space = {
    'n_estimators': np.linspace(10, 1000, num=20).astype('int'),
    'min_samples_split': np.linspace(2, 11, num=10).astype('int'),
    'max_features': ['sqrt', 'log2', None]
}

In [ ]:
max_score = 0
best_params = {}

for val in product(*search_space.values()):
    params = {}
    for i, param in enumerate(search_space.keys()):
        params[param] = val[i]
    print(params)

    pipeline = create_new_pipeline(params)

    pipeline.fit(X_train, y_train)

    score = pipeline.score(X_val, y_val)
    if score > max_score:
        max_score = score
        best_params = params
    print(f'Score: {score}\tBest score: {max_score}')

{'n_estimators': 10, 'min_samples_split': 2, 'max_features': 'sqrt'}
Score: 0.8249194414607949	Best score: 0.8249194414607949
{'n_estimators': 10, 'min_samples_split': 2, 'max_features': 'log2'}
Score: 0.8249194414607949	Best score: 0.8249194414607949
{'n_estimators': 10, 'min_samples_split': 2, 'max_features': None}
Score: 0.832438238453276	Best score: 0.832438238453276
{'n_estimators': 10, 'min_samples_split': 3, 'max_features': 'sqrt'}
Score: 0.8120300751879699	Best score: 0.832438238453276
{'n_estimators': 10, 'min_samples_split': 3, 'max_features': 'log2'}
Score: 0.8120300751879699	Best score: 0.832438238453276
{'n_estimators': 10, 'min_samples_split': 3, 'max_features': None}
Score: 0.8302900107411385	Best score: 0.832438238453276
{'n_estimators': 10, 'min_samples_split': 4, 'max_features': 'sqrt'}
Score: 0.8270676691729323	Best score: 0.832438238453276
{'n_estimators': 10, 'min_samples_split': 4, 'max_features': 'log2'}
Score: 0.8270676691729323	Best score: 0.832438238453276
{'n

In [ ]:
best_params

{'max_features': None, 'min_samples_split': 10, 'n_estimators': 62}

In [ ]:
max_score

0.8528464017185822

# Training

In [ ]:
pipeline = create_new_pipeline(best_params)

In [ ]:
pipeline.fit(X_full_train, y_full_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('numerical',
                                                  SimpleImputer(strategy='median'),
                                                  ['Age']),
                                                 ('categorical',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoding',
                                                                   OneHotEncoder(drop='first'))]),
                                                  ['Education', 'JoiningYear',
                                                   'City', 'PaymentTier',
                                                   'Gender', 'EverBenched',
                                                   'ExperienceInCurrentDomain'])])),
               

# Validation

In [ ]:
pipeline.score(X_full_train, y_full_train)

0.8903815153143472